In [1]:
#현재의 평균값
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
cap = cv2.VideoCapture('./data/vtest.avi')
if not cap.isOpened():
    print('Error~')
    
height, width = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))

In [3]:
TH = 80 #밝기값 임계값
AREA_TH = 100
mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE

acc_bgr = np.zeros(shape=(height, width, 3), dtype = np.float32)


In [7]:
t = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    t+=1
    blur = cv2.GaussianBlur(frame, (5, 5), .0) #delete noise
    if t < 50 : 
        cv2.accumulate(blur, acc_bgr)
        continue
    elif t == 50:
        bkg_bgr = acc_bgr / t
        
    diff_bgr = np.uint8(cv2.absdiff(np.float32(blur), bkg_bgr))
    
    diff_bgr = cv2.absdiff(frame, bkg_bgr) #스플릿
    db, dg, dr = cv2.split(diff_bgr) #채널별로 나눔
    ret, bb = cv2.threshold(db, TH, 255, cv2.THRESH_BINARY)
    ret, bg = cv2.threshold(dg, TH, 255, cv2.THRESH_BINARY)
    ret, br = cv2.threshold(dr, TH, 255, cv2.THRESH_BINARY)
    
    bimage = cv2.bitwise_or(bb, bg) #총합
    bimage = cv2.bitwise_or(br, bimage)
    
    bimage = cv2.erode(bimage, None, 5) #모폴로지 적용
    bimage = cv2.dilate(bimage, None, 5)
    bimage = cv2.erode(bimage, None, 7)
    
    cv2.imshow('bimage', bimage)
    
    contours, hierarcy = cv2.findContours(bimage, mode, method)
    cv2.drawContours(frame, contours, -1, (255, 0, 0), 1)
    for i, cnt in enumerate(contours):
        area = cv2.contourArea(cnt)
        if area > AREA_TH:
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x+w, x+h), (0, 0, 255), 2)

    cv2.imshow('frame', frame)
    cv2.imshow('bimage', bimage)
    cv2.imshow('diff_bgr', diff_bgr)
    
if cap.isOpened():
    cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\core\src\arithm.cpp:693: error: (-5:Bad argument) When the input arrays in add/subtract/multiply/divide functions have different types, the output array type must be explicitly specified in function 'cv::arithm_op'
